In [ ]:
import sys
import os
import cv2
import matplotlib.pyplot as plt
sys.path.append(os.path.abspath(os.path.join('..','src')))

from config import Config
from data_loader import DataLoader, ImagePreprocessing

from utils import Utils
from experiment import Reporting, Experiment

In [ ]:
# Get list of directories (classes)
config = Config('../src/config.json')
base_dir = config.get('data_path')
classes = os.listdir(base_dir)

# See classes in the dataset
print('Classes in the dataset:', classes)

In [ ]:
dataLoader = DataLoader(base_dir, 44)

df = dataLoader.create_dataframe(classes)

print(df.head())
print(df.tail())

In [ ]:
Utils.get_image_sizes_per_directory(df, base_dir)

In [ ]:
# Call the function with the DataFrame
Utils.display_dataset_images(df, base_dir, num_images_per_class=3)

In [ ]:
Utils.plot_image_distribution(df)

In [ ]:
def visualize_preprocessing(image_paths, num_images):
    """
    Visualize the images before and after preprocessing.

    This function loads a specified number of images, applies color segmentation
    and contour segmentation to each, and displays the original image, the 
    color-segmented image, and the contour-segmented image side by side.

    Parameters:
    - image_paths (list): List of paths to the images to be processed.
    - num_images (int): The number of images to display.

    Returns:
    - None
    """
    n_rows = num_images
    fig, axes = plt.subplots(n_rows, 3, figsize=(10, 2 * n_rows))
    
    # Set column titles
    column_titles = ['Original Image', 'Color-Segmented Image', 'Contour-Segmented Image']
    for ax, col_title in zip(axes[0], column_titles):
        ax.set_title(col_title, fontsize=16)
    
    for idx, image_path in enumerate(image_paths[:num_images]):
        # Load the original image
        original_image = cv2.imread(image_path)
        original_image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

        # Color segmented image
        color_segmented_image = ImagePreprocessing.segment_image_by_color(original_image_rgb)

        # Contour segmented image
        contour_segmented_image = ImagePreprocessing.segment_image_by_contour(original_image_rgb)

        # Display original image
        axes[idx, 0].imshow(original_image_rgb)
        axes[idx, 0].axis('off')

        # Display color-segmented image
        axes[idx, 1].imshow(color_segmented_image)
        axes[idx, 1].axis('off')

        # Display contour-segmented image
        axes[idx, 2].imshow(contour_segmented_image)
        axes[idx, 2].axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Visualize preprocessing
image_paths = [os.path.join(base_dir, path) for path in df.head()['path']]
visualize_preprocessing(image_paths, num_images=3)

In [ ]:
experiment = Experiment('../src/config.json')
all_results = experiment.run_experiment()

In [ ]:
reporting = Reporting()
reporting.plot_confusion_matrix_and_classification_report(all_results)

In [ ]:
experiment.save_model(all_results, 'DenseNet201', None, '0.7/0.15', '../model')